# VIDUS imputation
__Author__: Jesse Marks <br>

**GitHub Issue**: [#42](https://github.com/RTIInternational/bioinformatics/issues/42)

This document logs the steps taken to perform pre-imputation procedures on the [Vancouver Injection Drug Users Study (VIDUS)](https://www.bccsu.ca/vidus/) dataset, EAs. The starting point for this analysis is after quality control of observed genotypes. The quality controlled genotypes are oriented on the GRCh37 plus strand. 

## Software and tools
The software and tools used for porcessing these data are
* [Michigan Imputation Server](https://imputationserver.sph.umich.edu/index.html) (MIS)
* [Amazon Web Services (AWS) - Cloud Computing Services](https://aws.amazon.com/)
    * Linux AMI
* [PLINK v1.90 beta 4.10](https://www.cog-genomics.org/plink/)
* [bgzip](http://www.htslib.org/doc/tabix.html)
* [BCF Tools](http://www.htslib.org/doc/bcftools.html)
* Windows 10 with [Cygwin](https://cygwin.com/) installed
* GNU bash version 4.2.46

## Data retrieval and organization
PLINK binary filesets will be obtained from AWS S3 storage.

Nathan Gaddis performed the original QC and stored the observed genotype data at:

```
s3://rti-midas-data/studies/vidus/observed/final
```

These genotype data were QC'ed by Jesse Marks and stored on S3 at:

`s3://rti-hiv/vidus/data/genotype/observed/final`

## Create Directory Structure & Download Data
The following section needs to be modified each time to reflect where the data is stored!

**Note**: download genotype data for each study to respective data directories. For example, download STUDY genotype data (post-QC) to the appropriate directory: `${base_dir}/processing/STUDY/genotype/observed/$ancestry`

In [1]:
### EC2 command line (bash) ###

# Modify variables below
######################################################################
base_dir=/shared/jmarks/hiv/vidus/genotype/impute/processing/003
base_name="chr_all" # chr23 or chr_all
chr_list={1..23} # {1..23} or {1..22} 
ancestry_list="ea" # space delimited Ex. "ea aa ha"
study_list="vidus" # space delimited, lowercase
######################################################################

mkdir -p ${base_dir}/../final
mkdir ${base_dir}/{intersect,1000g,impute_prep} 
for study in ${study_list};do
    for ancestry in ${ancestry_list};do
        mkdir -p ${base_dir}/${study}/genotype/observed/${ancestry}
    done
done

## download genotype (with AWS CLI tools) to respective directories ##

# cd ${base_dir}/vidus/genotype/observed/ea/
# ln -s /shared/jmarks/hiv/vidus/genotype/observed/final/002/ea_chr_all.* .

# Data Processing
Make sure to merge autosomes with chrX for this pipeline.

## GRCh37 strand and allele discordance check
### MAF for study data

In [ ]:
### EC1 command line (Bash) ###

# write out the MAF report
for study in ${study_list}; do
    study_dir=${base_dir}/${study}/strand_check
    mkdir ${study_dir}
    for ancestry in ${ancestry_list};do
        data_dir=${base_dir}/${study}/genotype/observed/${ancestry}
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 2048 \
            --bed ${data_dir}/*bed\
            --bim ${data_dir}/*bim\
            --fam ${data_dir}/*fam\
            --freq \
            --out ${study_dir}/${ancestry}_${base_name}
    done
done

# Get list of variants from all studies
for ancestry in ${ancestry_list}; do
    for study in ${study_list};do
        cat ${base_dir}/${study}/genotype/observed/${ancestry}/*bim | \
                perl -lane 'if (($F[0]+0) <= 23) { print $F[1]; }' | \
                sort -u > ${base_dir}/${ancestry}_${base_name}_sorted_variants.txt
    done
done

```
wc -l ${base_dir}/*txt
700727 /shared/jmarks/heroin/vidus/genotype/impute/processing/002/ea_chr_all_sorted_variants.txt
```

### MAF for 1000G
This pipeline is currently set up to handle EUR and AFR populations.
#### Autosomes
Get 1000G MAF for chromosomes 1–22 (autosomes).

In [ ]:
### EC2 command line (Bash)

# Calculate autosome MAFs for 1000G populations
for ancestry in ${ancestry_list};do

    if [ $ancestry == "ea" ]
    then
        pop="EUR"
    else
        pop="AFR"
    fi
    
    for chr in {1..22}; do
        /shared/bioinformatics/software/scripts/qsub_job.sh \
            --job_name ${pop}_${chr}_MAF \
            --script_prefix ${base_dir}/1000g/${pop}_chr${chr}.maf \
            --mem 6.8 \
            --nslots 1 \
            --priority 0 \
            --program /shared/bioinformatics/software/perl/stats/calculate_maf_from_impute2_hap_file.pl \
                --hap /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chr${chr}.hap.gz\
                --legend /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chr${chr}.legend.gz \
                --sample /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3.sample \
                --chr ${chr} \
                --out ${base_dir}/1000g/${pop}_chr${chr}.maf \
                --extract ${base_dir}/${ancestry}_${base_name}_sorted_variants.txt \
                --keep_groups ${pop}
    done
done

#### chrX
Get 1000G MAF for chromosome 23 (chrX).

In [ ]:
### Bash ###

chr=23
for ancestry in ${ancestry_list};do

    if [ $ancestry == "ea" ]
    then
        pop="EUR"
    else
        pop="AFR"
    fi

    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ${pop}_23_MAF \
        --script_prefix ${base_dir}/1000g/${pop}_chr${chr}.maf \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program /shared/bioinformatics/software/perl/stats/calculate_maf_from_impute2_hap_file.pl \
            --hap /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chrX_NONPAR.hap.gz\
            --legend /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3_chrX_NONPAR.legend.gz \
            --sample /shared/data/ref_panels/1000G/2014.10/1000GP_Phase3.sample \
            --chr $chr \
            --out ${base_dir}/1000g/${pop}_chr${chr}.maf \
            --extract ${base_dir}/${ancestry}_${base_name}_sorted_variants.txt \
            --keep_groups ${pop}
done

### Merge 1000G chromosomes
Only need to perform this if there were multiple chromosomes for which the MAF was calculated—e.g. more than just chrX.

In [ ]:
### Bash ###

# Merge per chr MAFs for each 1000G population
for ancestry in ${ancestry_list};do
    if [ $ancestry == "ea" ]
    then
        pop="EUR"
    else
        pop="AFR"
    fi
    
        head -n 1 ${base_dir}/1000g/${pop}_chr1.maf > ${base_dir}/1000g/${pop}_chr_all.maf
        tail -q -n +2 ${base_dir}/1000g/${pop}_chr??maf  >> ${base_dir}/1000g/${pop}_chr_all.maf
done

```
wc -l ${base_dir}/1000g/???_chr_all.maf
/shared/jmarks/hiv/vidus/genotype/impute/processing/003/1000g/EUR_chr_all.maf
```

###  Allele Discordances Check
The allele discordances will be resolved by
* Flipping allele discordances that are fixed by flipping
* Removing SNPs with discordant names
* Removing SNPs with discordant positions
* Removing allele discordances that are not resolved by flipping
* Removing alleles with large deviations from the reference population allele frequencies

Given that the allele discordance check was done using a union set of SNPs across all studies within an ancestry group, some of the SNPs logged as discordant for a given study may not actually be in the study. Fortunately, if they are not in a given study they will not interfere with the filtering procedures. Note that the intersection set is used for the final studies merger.

#### Autosomes

In [ ]:
### Bash ###

chr="all"
# Run discordance checks for each ancestry group
for study in ${study_list}; do
    for ancestry in ${ancestry_list};do
        if [ $ancestry = "ea" ]; then
            pop=EUR
        else
            pop=AFR
        fi

       /shared/bioinformatics/software/scripts/qsub_job.sh \
           --job_name ${ancestry}_${study}_crosscheck_chr_${chr} \
           --script_prefix ${base_dir}/$study/strand_check/${ancestry}_allele_discordance_check \
           --mem 6 \
           --nslots 4 \
           --priority 0 \
           --program "Rscript /shared/bioinformatics/software/R/check_study_data_against_1000G.R
               --study_bim_file ${base_dir}/${study}/genotype/observed/${ancestry}/*bim
               --study_frq_file ${base_dir}/${study}/strand_check/${ancestry}_chr_all.frq
               --ref_maf_file ${base_dir}/1000g/${pop}_chr_all.maf
               --out_prefix ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance"
    done
done

#### chrX 
The following code should not be run unless chr23 was the only chromosome processed.

In [ ]:
#### Bash ###
#
#chr=23
#for study in ${study_list}; do
#    for ancestry in ${ancestry_list};do
#        if [ $ancestry = "ea" ]; then
#            pop=EUR
#        else
#            pop=AFR
#        fi
#
#        # chr23 discordance check
#        /shared/bioinformatics/software/scripts/qsub_job.sh \
#            --job_name ${ancestry}_${study}_crosscheck_chr${chr} \
#            --script_prefix ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance_check \
#            --mem 6.8 \
#            --nslots 1 \
#            --priority 0 \
#            --program "Rscript /shared/bioinformatics/software/R/check_study_data_against_1000G.R
#                --study_bim_file ${base_dir}/${study}/genotype/observed/${ancestry}/*bim
#                --study_frq_file ${base_dir}/${study}/strand_check/${ancestry}_chr23.frq
#                --ref_maf_file ${base_dir}/1000g/${pop}_chr23.maf
#                --out_prefix ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance_${chr}"
#    done
#done

### Resolving Allele Discordances

In [ ]:
### Bash ###

# Apply filters
for study in ${study_list}; do
    for ancestry in ${ancestry_list};do
        echo -e "\n===============\nProcessing ${study}_${ancestry}\n"
        echo "Making remove list"
        cat <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles_not_fixed_by_strand_flip | tail -n +2) \
            <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.at_cg_snps_freq_diff_gt_0.2 | tail -n +2) \
            <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.discordant_names | tail -n +2) \
            <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.discordant_positions | tail -n +2) \
            <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles_polymorphic_in_study_not_fixed_by_strand_flip | tail -n +2) | \
              sort -u > ${base_dir}/${study}/strand_check/${ancestry}_snps.remove

        # Create flip list
        echo "Making flip list"
        comm -23 <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles | tail -n +2 | sort -u) \
                 <(cut -f2,2 ${base_dir}/${study}/strand_check/${ancestry}_allele_discordance.discordant_alleles_not_fixed_by_strand_flip | tail -n +2 | sort -u) \
                 > ${base_dir}/${study}/strand_check/${ancestry}_snps.flip

        # Apply filters
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 2048 \
            --bed     ${base_dir}/${study}/genotype/observed/${ancestry}/*bed \
            --bim     ${base_dir}/${study}/genotype/observed/${ancestry}/*bim \
            --fam     ${base_dir}/${study}/genotype/observed/${ancestry}/*fam \
            --exclude ${base_dir}/${study}/strand_check/${ancestry}_snps.remove \
            --flip    ${base_dir}/${study}/strand_check/${ancestry}_snps.flip \
            --make-bed \
            --out     ${base_dir}/${study}/${ancestry}_filtered
    done
done

 ```
 wc -l ${base_dir}/*/*bim
 696828 /shared/jmarks/hiv/vidus/genotype/impute/processing/003/vidus/ea_filtered.bim
```

## Remove monomorphic variants
Monomorphic variants prevent MIS from accepting the genotype data. In this case, an arbitrarily small MAF is set that is smaller than the lower bound for these data.

In [ ]:
### Bash ###

# Apply filters
for study in ${study_list}; do
    for ancestry in ${ancestry_list};do
        geno_dir=${base_dir}/${study}

        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 2048 \
            --bfile ${geno_dir}/${ancestry}_filtered \
            --maf 0.000001 \
            --make-bed \
            --out ${geno_dir}/${ancestry}_filtered_mono
    done
done

```
 wc -l ${base_dir}/*/*mono.bim
 /shared/jmarks/hiv/vidus/genotype/impute/processing/003/vidus/ea_filtered_mono.bim
```

### SNP intersection
Only run if merging multiple data sets.

In [ ]:
#studies=($study_list)  #studies=(UHS1 UHS2 UHS3_v1-2 UHS3_v1-3) # array of study names
#num=${#studies[@]}
#
## Get intersection set
#for ancestry in ${ancestry_list};do
#    bim_files=()
#    for (( i=0; i<${num}; i++ ));do
#        bim_files+=(${base_dir}/processing/${studies[$i]}/${ancestry}_filtered_mono.bim)
#    done
#    
#    echo -e "\nCalculating intersection between $ancestry ${study_list}...\n"
#    sort ${bim_files[@]} | uniq -dc | awk -v num=$num '$1 == num {print $3}' \
#        > ${base_dir}/processing/intersect/${ancestry}_variant_intersection.txt
#
#    # Make new PLINK binary file sets
#    for study in ${studies[@]}; do
#        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
#            --noweb \
#            --bfile ${base_dir}/processing/${study}/${ancestry}_filtered_mono \
#            --extract ${base_dir}/processing/intersect/${ancestry}_variant_intersection.txt \
#            --make-bed \
#            --out ${base_dir}/processing/intersect/${study}_${ancestry}_filtered_snp_intersection
#    done
#done

```wc -l *txt

```

### Merge test
If merging multiple datasets together:

As a final check to confirm that our data sets are all compatible, a PLINK file set merge is conducted. If any issues persist then an error will be raised.

In [ ]:
#for ancestry in $ancestry_list;do
#
#    echo "Creating $ancestry merge-list"
#    touch ${base_dir}/processing/intersect/${ancestry}_merge_list.txt
#    for study in $study_list;do
#        echo ${base_dir}/processing/intersect/${study}_${ancestry}_filtered_snp_intersection >>\
#             ${base_dir}/processing/intersect/${ancestry}_merge_list.txt
#    done
#
## Merge file sets
#    echo -e "\n\n======== ${ancestry} ========\n\n"
#    /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
#        --noweb \
#        --memory 4000 \
#        --merge-list ${base_dir}/processing/intersect/${ancestry}_merge_list.txt \
#        --make-bed \
#        --out ${base_dir}/processing/intersect/${ancestry}_studies_merged
#done

```
 wc -l *merged.bim
```

## Imputation preparation for Michigan Imputation Server
Visit the [MIS Getting Started Webpage](https://imputationserver.sph.umich.edu/start.html#!pages/help) for more information about the preparing the data for upload to MIS.


### VCF File Conversion

#### If merging was done

In [ ]:
### Split by chr and remove any individuals with missing data for whole chr

## if merged data sets together
#for ancestry in $ancestry_list;do
#    # Remove SNPs
#    /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
#        --noweb \
#        --memory 4000 \
#        --bfile ${base_dir}/processing/intersect/${ancestry}_studies_merged \
#        --chr ${chr} \
#        --mind 0.99 \
#        --make-bed \
#        --out ${base_dir}/processing/impute_prep/${ancestry}_chr${chr}_for_phasing 
#done > ${base_dir}/processing/impute_prep/chr_splitting.log 

#### If NO merging was performed 
I.e. if there was only one data set being processing.

In [ ]:
## if NO merging was done
for ancestry in $ancestry_list;do
    for chr in {1..23};do
        # Remove SNPs
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 4000 \
            --bfile ${base_dir}/$study/${ancestry}_filtered_mono \
            --chr ${chr} \
            --mind 0.99 \
            --make-bed \
            --out ${base_dir}/impute_prep/${ancestry}_chr${chr}_for_phasing 
    done
done > ${base_dir}/impute_prep/chr_splitting.log 

__Note__: No subjects were removed.

```
grep removed ${base_dir}/impute_prep/chr_splitting.log
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
0 people removed due to missing genotype data (--mind).
```

In [ ]:
# EC2 command line #

for chr in {1..23};do
    for ancestry in ${ancestry_list};do
        final_dir=${base_dir}/impute_prep/${ancestry}
        mkdir $final_dir
        /shared/bioinformatics/software/third_party/plink-1.90-beta-4.10-x86_64/plink \
            --noweb \
            --memory 5000 \
            --bfile ${final_dir}/../${ancestry}_chr${chr}_for_phasing \
            --output-chr M \
            --set-hh-missing \
            --recode vcf bgz \
            --out ${final_dir}/${ancestry}_chr${chr}_final
    done
done

Transfer the *.vcf.gz files to local machine (per chromosome) and then upload to MIS.
```
scp -i ~/.ssh/gwas_rsa ec2-user@34.206.166.72:/shared/jmarks/hiv/vidus/genotype/impute/processing/003/impute_prep/ea/*gz .
```

## Upload to Michigan Imputation Server (MIS)
Note, it is best to upload the autosomes and chrX independently from each other; if one chromosome crashes then the entire job will fail.

### Uploading parameters
These are the parameters that were selected on MIS.

__Name__: VIDUS_marks_qc_20190208

__Reference Panel__ 1000G Phase 3 v5

__Input Files__ File Upload <br>

* Select Files - select VCF (.gz) files that were downloaded to local machine from cloud. <br>

__Phasing__: ShapeIT v2.r790 (unphased) 

__Population__: EUR

__Mode__: Quality Control & Imputation

* I will not attempt to re-identify or contact research participants.
* I will report any inadvertent data release, security breach or other data management incident of which I become aware.

# Download Imputed Results from MIS
First, download the data form the Michigan Imputation Server (MIS) by clicking on the link provided in the email they send out to alert you that your data has finished. Here you will find commands for downloading the data.

The zip files from Michigan Imputation Server (MIS) need to be inflated before you can begin working with them. They require a passcode that is sent by MIS to email.

Then, inflate imputation results.

## EA
### Autosomes

In [ ]:
ancestry=ea
passW="i2ikMTaOUj9Td$"
cd /shared/jmarks/hiv/vidus/genotype/impute/final/$ancestry/auto

# bash script
################################################################################
# QC-results
 wget https://imputationserver.sph.umich.edu/share/results/db6ff65759a52629a98a8ef7347e1cae/qcreport.html

# SNP Statistics
 wget https://imputationserver.sph.umich.edu/share/results/9833912b9bca5c2d4ef6f4f0c56f76ce/statistics.txt


# Logs
wget https://imputationserver.sph.umich.edu/share/results/d16683383b897f0595fa699ab4701d49/chr_1.log
wget https://imputationserver.sph.umich.edu/share/results/61324872ab49c6258854865e0dd97983/chr_10.log
wget https://imputationserver.sph.umich.edu/share/results/1f2ec453b2fe2c5c66951c558b321529/chr_11.log
wget https://imputationserver.sph.umich.edu/share/results/a2b4613ffab2dea702cf557f5be5df9f/chr_12.log
wget https://imputationserver.sph.umich.edu/share/results/8dbc8e218f3fdd0d8daff70ec9584bbb/chr_13.log
wget https://imputationserver.sph.umich.edu/share/results/da09ded63240bc7ec1b25d1499d327be/chr_14.log
wget https://imputationserver.sph.umich.edu/share/results/e085389e514d5ca45b26e7ab1f7a4aa6/chr_15.log
wget https://imputationserver.sph.umich.edu/share/results/86618e84bdf8b66e3a5574fee3aebd14/chr_16.log
wget https://imputationserver.sph.umich.edu/share/results/d8b0b924435442589d7b35bd84b0c256/chr_17.log
wget https://imputationserver.sph.umich.edu/share/results/a814fd0edd6ed2c4903b74eb7af522b8/chr_18.log
wget https://imputationserver.sph.umich.edu/share/results/de884f09772f691fe64fb6a7a804fd83/chr_19.log
wget https://imputationserver.sph.umich.edu/share/results/a1bf834909b0ab47b1e0984f1b29004d/chr_2.log
wget https://imputationserver.sph.umich.edu/share/results/5b6d83f20722c7bcab28d3f4584319fc/chr_20.log
wget https://imputationserver.sph.umich.edu/share/results/8124b8f60738b484441688d948fb7c93/chr_21.log
wget https://imputationserver.sph.umich.edu/share/results/29097fef87f5a487042e7cec9ce557c1/chr_22.log
wget https://imputationserver.sph.umich.edu/share/results/b6cf95751033f16d29d48d452606fac/chr_3.log
wget https://imputationserver.sph.umich.edu/share/results/9395b19aa1698836c160b1d2c3474db1/chr_4.log
wget https://imputationserver.sph.umich.edu/share/results/4a2cf45d3b20230ab636b604631ca900/chr_5.log
wget https://imputationserver.sph.umich.edu/share/results/3ba6c0a459147b02808d335477e2ed8a/chr_6.log
wget https://imputationserver.sph.umich.edu/share/results/297f42cafe630f4d32353580061fe59a/chr_7.log
wget https://imputationserver.sph.umich.edu/share/results/6988498cd4f208740b2c11fbacb2793d/chr_8.log
wget https://imputationserver.sph.umich.edu/share/results/c89c83929edeae6673dba52c0bea84fd/chr_9.log


# Imputation Results
wget https://imputationserver.sph.umich.edu/share/results/58a7777e1e46e5a3f8ec4c82c28bc443/chr_1.zip
wget https://imputationserver.sph.umich.edu/share/results/f80a30a996475edf9ed239cbf9d92ba1/chr_10.zip
wget https://imputationserver.sph.umich.edu/share/results/17ef033d634dc1c5180e05060a0adbf9/chr_11.zip
wget https://imputationserver.sph.umich.edu/share/results/bc7aed458198db05edc8892db1d17b45/chr_12.zip
wget https://imputationserver.sph.umich.edu/share/results/2e175b75021f2fd4130accf0e937fba3/chr_13.zip
wget https://imputationserver.sph.umich.edu/share/results/3d80c89175480da4f8da07d149f9e2bd/chr_14.zip
wget https://imputationserver.sph.umich.edu/share/results/4887fa0c9524663e52c713e07d885488/chr_15.zip
wget https://imputationserver.sph.umich.edu/share/results/58174f6a3dad7f72b083a4144d42a16b/chr_16.zip
wget https://imputationserver.sph.umich.edu/share/results/935a4f7dfeb5e80f2297b57b30539fd1/chr_17.zip
wget https://imputationserver.sph.umich.edu/share/results/e26d8cfadbf6ec8e55afb6401af10f6e/chr_18.zip
wget https://imputationserver.sph.umich.edu/share/results/7b70f0e6f24eb7dd28a1ed4e12445d65/chr_19.zip
wget https://imputationserver.sph.umich.edu/share/results/bd9342657a0f3e28137fd02f6b7aae3a/chr_2.zip
wget https://imputationserver.sph.umich.edu/share/results/79b9444b0cac767b6cd3f4a0dacd1f9c/chr_20.zip
wget https://imputationserver.sph.umich.edu/share/results/5fd700d2cc141735603be99f9329a534/chr_21.zip
wget https://imputationserver.sph.umich.edu/share/results/c1375f9e6cc0c96e8e0b34d8dcb50dff/chr_22.zip
wget https://imputationserver.sph.umich.edu/share/results/c191cdd8689bd9457c7df8e5e3aaac00/chr_3.zip
wget https://imputationserver.sph.umich.edu/share/results/77219af4da5fa39575fd038f7da769a4/chr_4.zip
wget https://imputationserver.sph.umich.edu/share/results/2ef544943ae73539fe8f950af8e96cf/chr_5.zip
wget https://imputationserver.sph.umich.edu/share/results/f372dda74f12b5651b7bda7cd108ed5a/chr_6.zip
wget https://imputationserver.sph.umich.edu/share/results/aab623d6a9116135ad977103baf3150c/chr_7.zip
wget https://imputationserver.sph.umich.edu/share/results/3ba0c3646d2b0d890e324cbb329f7460/chr_8.zip
wget https://imputationserver.sph.umich.edu/share/results/af952aef884a234569740fcce3421259/chr_9.zip


################################################################################


# paste code above into bash script 
/shared/bioinformatics/software/scripts/qsub_job.sh \
    --job_name ${ancestry}_impute_down_auto \
    --script_prefix ${ancestry}_impute_download_auto \
    --mem 6.8 \
    --nslots 1 \
    --priority 0 \
    --program bash download_auto

# inflate chr results
for file in *zip;do
    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ${ancestry}_unzip_auto \
        --script_prefix ${ancestry}_unzip_auto \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program unzip -P $passW $file
done
rm -rf *zip

### chrX 
The zip files from Michigan Imputation Server (MIS) need to be inflated before you can begin working with them. They require a passcode that is sent by MIS to email.

Then, inflate imputation results.

In [ ]:
passW="omZS0WOh)VdeT2"
ancestry=ea
cd /shared/jmarks/hiv/vidus/genotype/impute/final/$ancestry/chrx

# bash script
################################################################################
# QC-results
 wget https://imputationserver.sph.umich.edu/share/results/bd59e47457e5ef3e2df0276f7468a822/qcreport.html

# SNP Statistics
 wget https://imputationserver.sph.umich.edu/share/results/859e69d7b121decef8cadd9c438678fc/statistics.txt

# Logs
wget https://imputationserver.sph.umich.edu/share/results/f64a6a8d68d20321986ba81e195e2e6/chr_X.no.auto_female.log
wget https://imputationserver.sph.umich.edu/share/results/cb9d2166556b549cdeafc5ff762aeef6/chr_X.no.auto_male.log

# Imputation Results
wget https://imputationserver.sph.umich.edu/share/results/12957a26892de47840e5d1f592bc1f04/chr_X.no.auto_female.zip
wget https://imputationserver.sph.umich.edu/share/results/b4a1fc9cc75dcf2a2a83f1902356aad4/chr_X.no.auto_male.zip
################################################################################


# paste code above into bash script called download_chrx
/shared/bioinformatics/software/scripts/qsub_job.sh \
    --job_name ${ancestry}_impute_down \
    --script_prefix ${ancestry}_impute_download \
    --mem 6.8 \
    --nslots 1 \
    --priority 0 \
    --program bash download_chrx

# inflate chr results
for file in *zip;do
    /shared/bioinformatics/software/scripts/qsub_job.sh \
        --job_name ${ancestry}_unzip_chrx \
        --script_prefix ${ancestry}_unzip_chrx \
        --mem 6.8 \
        --nslots 1 \
        --priority 0 \
        --program unzip -P $passW $file
done

rm -rf *zip

# Upload to S3
## autosomes

In [ ]:
cd /shared/jmarks/hiv/vidus/genotype/impute/final/ea/auto

/shared/bioinformatics/software/scripts/qsub_job.sh \
    --job_name ${ancestry}_upload_auto \
    --script_prefix ${ancestry}_upload_auto \
    --mem 6.8 \
    --nslots 1 \
    --priority 0 \
    --program aws s3 sync . s3://rti-hiv/vidus/data/genotype/imputed/

## chrx

In [ ]:
cd /shared/jmarks/hiv/vidus/genotype/impute/final/ea/chrx

/shared/bioinformatics/software/scripts/qsub_job.sh \
    --job_name ${ancestry}_upload_chrx \
    --script_prefix ${ancestry}_upload_chrx \
    --mem 6.8 \
    --nslots 1 \
    --priority 0 \
    --program aws s3 sync . s3://rti-hiv/vidus/data/genotype/imputed/